In [1]:
# ============================================ Bias Analyzer (Colab-ready)
# ============================================

# ---- Auto install dependencies (safe & idempotent) ----
import sys, subprocess, importlib

def ensure(pkg, import_name=None):
    import_name = import_name or pkg
    try:
        importlib.import_module(import_name)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", pkg])
        importlib.invalidate_caches()

# Core libs
ensure("torch")
ensure("transformers")
ensure("numpy")
ensure("scipy")
ensure("nltk")
# Detoxify for toxicity (has CPU-only fallback if no GPU)
ensure("detoxify")

import re
import math
import numpy as np
from statistics import mean, pstdev
from dataclasses import dataclass
from typing import List, Dict, Any

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from detoxify import Detoxify
import nltk

# NLTK lightweight setup (we avoid punkt to keep footprint small)
# For sentiment we need vader_lexicon only
try:
    nltk.data.find("sentiment/vader_lexicon.zip")
except LookupError:
    nltk.download("vader_lexicon", quiet=True)

from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# ---------------- Utilities ----------------

def split_sentences(text: str) -> List[str]:
    # Lightweight sentence splitter to avoid large NLTK punkt
    # Splits on .?! while preserving reasonable tokens
    raw = re.split(r'(?<=[\.\?\!])\s+', text.strip())
    # Clean and filter
    return [s.strip() for s in raw if s.strip()]

def tokenize_words(text: str) -> List[str]:
    return re.findall(r"[A-Za-zÀ-ÖØ-öø-ÿ0-9']+", text.lower())

def type_token_ratio(tokens: List[str]) -> float:
    if not tokens:
        return 0.0
    return len(set(tokens)) / max(1, len(tokens))

def repetition_ratio(tokens: List[str], n: int = 3) -> float:
    """
    Measures n-gram repetition (0..1). Higher -> more repetition (AI often repeats).
    """
    if len(tokens) < n:
        return 0.0
    ngrams = [" ".join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
    if not ngrams:
        return 0.0
    unique = len(set(ngrams))
    return 1.0 - (unique / len(ngrams))

def sentence_length_stats(sentences: List[str]) -> Dict[str, float]:
    lengths = [len(tokenize_words(s)) for s in sentences if s.strip()]
    if not lengths:
        return {"avg": 0.0, "std": 0.0, "cv": 0.0}
    avg = mean(lengths)
    std = pstdev(lengths) if len(lengths) > 1 else 0.0
    cv = (std / avg) if avg > 0 else 0.0
    return {"avg": float(avg), "std": float(std), "cv": float(cv)}

def softclip(x, lo, hi):
    return max(lo, min(hi, x))

# ---------------- Perplexity via GPT-2 ----------------
# We use GPT-2 to compute perplexity (lower perplexity -> more "model-like")
# Robustly handles long texts by sliding window.
class PerplexityScorer:
    def __init__(self, model_name: str = "gpt2", max_length: int = 1024, stride: int = 512, device: str = None):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = device
        self.model.to(self.device)
        self.model.eval()
        self.max_length = max_length
        self.stride = stride

    @torch.no_grad()
    def perplexity(self, text: str) -> float:
        if not text.strip():
            return float("inf")
        encodings = self.tokenizer(text, return_tensors="pt")
        input_ids = encodings.input_ids.to(self.device)
        nlls = []
        for i in range(0, input_ids.size(1), self.stride):
            begin_loc = max(i + self.stride - self.max_length, 0)
            end_loc = min(i + self.stride, input_ids.size(1))
            trg_len = end_loc - i
            if trg_len <= 0:
                continue
            input_ids_slice = input_ids[:, begin_loc:end_loc]
            target_ids = input_ids_slice.clone()
            target_ids[:, :-trg_len] = -100  # mask everything except target region
            outputs = self.model(input_ids_slice, labels=target_ids)
            nll = outputs.loss * trg_len
            nlls.append(nll)
        if not nlls:
            return float("inf")
        ppl = torch.exp(torch.stack(nlls).sum() / max(1, input_ids.size(1))).item()
        # Guard against numeric issues
        if math.isinf(ppl) or math.isnan(ppl):
            return 1e6
        return float(ppl)

# ---------------- Toxicity & Sentiment ----------------

class RobustDetoxify:
    """
    Wrap Detoxify with CPU fallback and graceful degradation.
    """
    def __init__(self):
        # Use 'original' weights (multi-label) suitable for general toxicity
        try:
            self.model = Detoxify('original')
            self.ok = True
        except Exception:
            self.model = None
            self.ok = False

    def score(self, text: str) -> Dict[str, float]:
        if not self.ok or not text.strip():
            return {"toxicity": 0.0, "severe_toxicity": 0.0, "obscene": 0.0,
                    "threat": 0.0, "insult": 0.0, "identity_attack": 0.0}
        try:
            out = self.model.predict(text)
            # Ensure presence of keys
            for k in ["toxicity","severe_toxicity","obscene","threat","insult","identity_attack"]:
                if k not in out:
                    out[k] = 0.0
            return {k: float(out[k]) for k in out}
        except Exception:
            return {"toxicity": 0.0, "severe_toxicity": 0.0, "obscene": 0.0,
                    "threat": 0.0, "insult": 0.0, "identity_attack": 0.0}

# ---------------- Bias Lexicon ----------------

PROTECTED_TERMS = set([
    # Gender & identity
    "woman","women","man","men","female","male","nonbinary","trans","transgender","lgbt","gay","lesbian","bisexual","queer",
    # Race/ethnicity (examples; non-exhaustive, lowercase)
    "black","white","asian","latino","hispanic","arab","indian","african","european","indigenous","native","jewish","jews","muslim","christian","christians",
    # Religion (more)
    "islam","christianity","judaism","buddhist","hindu","sikh",
    # Age/disability
    "disabled","disability","handicapped","elderly","old","young","youth","senior",
    # Nationality (examples)
    "american","british","irish","french","german","italian","spanish","russian","ukrainian","chinese","japanese","korean","turkish","saudi","emirati","egyptian","jordanian","moroccan"
])

NEGATION_TERMS = set(["not","never","no","none","hardly","barely","without","less"])

def contains_protected_terms(text: str) -> bool:
    tokens = tokenize_words(text)
    return any(t in PROTECTED_TERMS for t in tokens)

# ---------------- Feature Engineering ----------------

@dataclass
class TextFeatures:
    perplexity: float
    ttr: float
    repetition: float
    sent_cv: float
    avg_sent_len: float

def extract_features(text: str, ppl_scorer: PerplexityScorer) -> TextFeatures:
    sentences = split_sentences(text)
    tokens = tokenize_words(text)
    stats = sentence_length_stats(sentences)
    ppl = ppl_scorer.perplexity(text)
    return TextFeatures(
        perplexity=ppl,
        ttr=type_token_ratio(tokens),
        repetition=repetition_ratio(tokens, n=3),
        sent_cv=stats["cv"],
        avg_sent_len=stats["avg"]
    )

# ---------------- AI Probability (Heuristic Model) ----------------
# We map features to a probability via a simple, well-behaved logistic model.
# Intuition:
#   • Lower perplexity  -> more AI-like
#   • Lower burstiness (CV) -> more AI-like
#   • Higher repetition -> more AI-like
#   • Lower TTR         -> more AI-like

def sigmoid(x): return 1 / (1 + math.exp(-x))

def ai_probability(feat: TextFeatures) -> float:
    # Normalize features to stable ranges
    # Perplexity is unbounded & skewed; clip to [5, 200] then invert.
    ppl = softclip(feat.perplexity, 5, 200)
    ppl_norm = (ppl - 5) / (200 - 5)  # 0 (very low ppl) -> 1 (high ppl)
    # We want lower perplexity => higher AI-ness, so invert:
    ppl_ai = 1.0 - ppl_norm

    # CV (0..2 typical); cap at 1.5
    cv = softclip(feat.sent_cv, 0.0, 1.5) / 1.5
    cv_ai = 1.0 - cv  # lower CV -> more AI-like

    # Repetition already 0..1 (higher -> AI-like)
    rep_ai = softclip(feat.repetition, 0.0, 1.0)

    # TTR (0..1) (lower -> AI-like)
    ttr_ai = 1.0 - softclip(feat.ttr, 0.0, 1.0)

    # Weighted sum -> logistic
    # Weights tuned for balanced influence
    z = (2.2 * ppl_ai) + (1.6 * cv_ai) + (1.2 * rep_ai) + (1.0 * ttr_ai) - 2.0
    return float(softclip(sigmoid(z), 0.01, 0.99))

# ---------------- Mixed Authorship Detection ----------------

def segment_ai_mix(text: str, ppl_scorer: PerplexityScorer) -> Dict[str, Any]:
    sentences = split_sentences(text)
    if not sentences:
        return {"ai_like": 0.0, "human_like": 0.0, "label": "insufficient_text"}
    # Group into chunks of ~4 sentences for stable scoring
    chunks = []
    for i in range(0, len(sentences), 4):
        chunk = " ".join(sentences[i:i+4])
        if chunk.strip():
            chunks.append(chunk)
    probs = []
    for ch in chunks:
        f = extract_features(ch, ppl_scorer)
        probs.append(ai_probability(f))
    if not probs:
        return {"ai_like": 0.0, "human_like": 0.0, "label": "insufficient_text"}
    ai_frac = float(np.mean([p > 0.6 for p in probs]))
    human_frac = float(np.mean([p < 0.4 for p in probs]))
    if ai_frac >= 0.3 and human_frac >= 0.3:
        label = "mixed"
    elif ai_frac >= 0.5:
        label = "ai_generated_likely"
    elif human_frac >= 0.5:
        label = "human_written_likely"
    else:
        label = "unclear"
    return {"ai_like": ai_frac, "human_like": human_frac, "label": label, "chunk_probs": probs}

# ---------------- Bias Scoring ----------------

def bias_scores(text: str, tox_model: RobustDetoxify) -> Dict[str, Any]:
    # Toxicity (0..1 for each)
    tox = tox_model.score(text)
    toxicity = float(tox.get("toxicity", 0.0))
    identity_attack = float(tox.get("identity_attack", 0.0))
    insult = float(tox.get("insult", 0.0))
    severe = float(tox.get("severe_toxicity", 0.0))
    threat = float(tox.get("threat", 0.0))

    # Sentiment extremity (abs compound)
    sent = sia.polarity_scores(text)
    sentiment_extremity = abs(float(sent.get("compound", 0.0)))  # 0..1

    # Mentions of protected groups
    has_protected = contains_protected_terms(text)

    # Bias aggregator:
    # Base toxicity weight
    base = 0.5 * toxicity + 0.1 * severe + 0.1 * threat + 0.15 * insult + 0.15 * identity_attack
    # If protected groups mentioned, amplify by sentiment extremity (negative contexts tend to be higher risk)
    protected_amp = (0.4 * sentiment_extremity) if has_protected else (0.15 * sentiment_extremity)
    score = softclip(base + protected_amp, 0.0, 1.0)

    # Map to 0..100
    overall = round(100.0 * score, 2)

    return {
        "overall_bias_score_0_100": overall,
        "toxicity": round(100 * toxicity, 2),
        "identity_attack": round(100 * identity_attack, 2),
        "insult": round(100 * insult, 2),
        "severe_toxicity": round(100 * severe, 2),
        "threat": round(100 * threat, 2),
        "sentiment_extremity_0_100": round(100 * sentiment_extremity, 2),
        "mentions_protected_groups": bool(has_protected)
    }

# ---------------- Main API ----------------

def analyze_text(text: str) -> Dict[str, Any]:
    text = (text or "").strip()
    if not text:
        return {"error": "Empty text. Please provide non-empty input."}

    # Init scorers
    ppl_scorer = PerplexityScorer(model_name="gpt2", max_length=1024, stride=512)
    tox_model = RobustDetoxify()

    # Global features & AI probability
    features = extract_features(text, ppl_scorer)
    ai_prob = ai_probability(features)
    mix = segment_ai_mix(text, ppl_scorer)

    # Labeling
    if mix["label"] == "mixed":
        label = "mixed_authorship_suspected"
    else:
        if ai_prob >= 0.65:
            label = "ai_generated_likely"
        elif ai_prob <= 0.35:
            label = "human_written_likely"
        else:
            label = "unclear"

    # Bias scoring
    bias = bias_scores(text, tox_model)

    # Package results
    return {
        "ai_detection": {
            "ai_probability_0_1": round(ai_prob, 4),
            "label": label,
            "global_features": {
                "perplexity": round(features.perplexity, 2),
                "type_token_ratio": round(features.ttr, 4),
                "repetition_3gram": round(features.repetition, 4),
                "sentence_length_cv": round(features.sent_cv, 4),
                "avg_sentence_length_tokens": round(features.avg_sent_len, 2),
            },
            "segment_mix_analysis": {
                "ai_like_fraction": round(float(mix.get("ai_like", 0.0)), 3),
                "human_like_fraction": round(float(mix.get("human_like", 0.0)), 3),
                "segment_label": mix.get("label", "n/a"),
                "segment_ai_probs": [round(float(p), 3) for p in mix.get("chunk_probs", [])]
            }
        },
        "bias_assessment": bias
    }

# ---------------- Example Usage ----------------
if __name__ == "__main__":
    # You can replace this sample with any text, or use input() for quick testing.
    sample_text = (
        "Artificial intelligence models have transformed how we summarize and write content. "
        "This paragraph is intentionally neutral. However, claims that any specific nationality is inherently better at science "
        "are misguided and not supported by evidence."
    )

    print("=== AI & Bias Analyzer ===")
    print("Provide your text below. Press Enter to use the built-in sample.\n")
    try:
        user_inp = input("Paste text (or press Enter to analyze the sample): ").strip()
    except EOFError:
        user_inp = ""

    text = user_inp if user_inp else sample_text
    results = analyze_text(text)
    import json
    print(json.dumps(results, indent=2, ensure_ascii=False))


=== AI & Bias Analyzer ===
Provide your text below. Press Enter to use the built-in sample.

Paste text (or press Enter to analyze the sample): Hi my name is shahed i am 29


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


{
  "ai_detection": {
    "ai_probability_0_1": 0.6469,
    "label": "unclear",
    "global_features": {
      "perplexity": 110.89,
      "type_token_ratio": 1.0,
      "repetition_3gram": 0.0,
      "sentence_length_cv": 0.0,
      "avg_sentence_length_tokens": 8.0
    },
    "segment_mix_analysis": {
      "ai_like_fraction": 1.0,
      "human_like_fraction": 0.0,
      "segment_label": "ai_generated_likely",
      "segment_ai_probs": [
        0.647
      ]
    }
  },
  "bias_assessment": {
    "overall_bias_score_0_100": 0.06,
    "toxicity": 0.1,
    "identity_attack": 0.01,
    "insult": 0.02,
    "severe_toxicity": 0.01,
    "threat": 0.01,
    "sentiment_extremity_0_100": 0.0,
    "mentions_protected_groups": false
  }
}
